In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import copy # 导入copy模块，用于深拷贝颜色字典

# ==============================================================================
# 1. 环境设置 (解决中文显示问题)
# ==============================================================================
try:
    # 优先使用更现代且跨平台性更好的黑体
    plt.rcParams['font.sans-serif'] = ['Heiti TC', 'Heiti SC', 'PingFang SC', 'SimHei', 'Arial Unicode MS']
    plt.rcParams['axes.unicode_minus'] = False
    plt.rcParams['svg.fonttype'] = 'none' # 确保SVG文件中的字体是可编辑的
except Exception as e:
    print(f"注意：未能设置中文字体。图表中的中文可能无法正常显示。错误信息: {e}")

# ==============================================================================
# 2. 创建你的数据表
# ==============================================================================
data = {
    'Model': [
        '1. FP32 Baseline (12L)',
        '2. INT4 BitsAndBytes (12L, GPU-Only)',
        '3. INT8 PTQ (12L, CPU-Only)',
        '4. INT8 QAT (12L)',
        '5. Pruned FP32 (8L)',
        '6. Pruned FP16 (8L, GPU-Only)'
    ],
    'Size (MB)': [1253.16, 91.64, 173.09, 418.63, 310.42, 155.66],
    'Accuracy (GPU)': [0.9300, 0.9300, np.nan, 0.9255, 0.9278, 0.9266],
    'Latency (GPU, ms)': [3.23, 8.92, np.nan, 3.22, 2.30, 2.28],
    'Peak GPU Mem (MB)': [428.26, 106.13, np.nan, 428.56, 320.98, 169.25],
    'Accuracy (CPU)': [0.9300, np.nan, 0.9186, 0.9255, 0.9278, np.nan],
    'Latency (CPU, ms)': [127.99, np.nan, 67.03, 132.04, 114.94, np.nan]
}
df = pd.DataFrame(data)

# ==============================================================================
# 3. 数据预处理：归一化
# ==============================================================================
benefit_metrics = ['Accuracy (GPU)', 'Accuracy (CPU)']
cost_metrics = ['Size (MB)', 'Latency (GPU, ms)', 'Peak GPU Mem (MB)', 'Latency (CPU, ms)']
df_normalized = df.copy()
alpha = 0.1

semantic_min_acc = 0.90
semantic_max_acc = 0.94

for col in benefit_metrics + cost_metrics:
    if col in benefit_metrics:
        min_val = semantic_min_acc
        max_val = semantic_max_acc
        range_val = max_val - min_val
        clipped_values = df_normalized[col].clip(lower=min_val)
        normalized_values = alpha + (1 - alpha) * (clipped_values - min_val) / range_val
        df_normalized[col] = normalized_values.clip(upper=1.0)
    
    elif col in cost_metrics:
        min_val = df_normalized[col].min()
        max_val = df_normalized[col].max()
        if pd.isna(min_val) or pd.isna(max_val) or min_val == max_val:
            if not pd.isna(min_val):
                df_normalized[col] = 0.5
            continue
        range_val = max_val - min_val
        df_normalized[col] = alpha + (1 - alpha) * (max_val - df_normalized[col]) / range_val

# ==============================================================================
# 4. 分离GPU和CPU数据
# ==============================================================================
gpu_models_df = df_normalized.dropna(subset=['Accuracy (GPU)']).reset_index(drop=True)
cpu_models_df = df_normalized.dropna(subset=['Accuracy (CPU)']).reset_index(drop=True)

gpu_metrics = ['Size (MB)', 'Accuracy (GPU)', 'Latency (GPU, ms)', 'Peak GPU Mem (MB)']
cpu_metrics = ['Size (MB)', 'Accuracy (CPU)', 'Latency (CPU, ms)']

# ==============================================================================
# 5. 定义基础配色方案
# ==============================================================================
# 使用您选定的学术化配色方案作为基础
model_colors = {
    '1. FP32 Baseline (12L)': '#888888',                 # 中性灰色 (Neutral Grey)
    '2. INT4 BitsAndBytes (12L, GPU-Only)': '#0173B2',   # 稳重蓝色 (Calm Blue)
    '3. INT8 PTQ (12L, CPU-Only)': '#029E73',            # 青绿色 (Teal Green)
    '4. INT8 QAT (12L)': '#DE8F05',                      # 赭石色 (Ochre)
    '5. Pruned FP32 (8L)': '#D55E00',                    # 朱红色/强调色 (Vermilion - Highlight)
    '6. Pruned FP16 (8L, GPU-Only)': '#CC78BC'           # 品红色/强调色 (Magenta - Highlight)
}

# 定义你想要高亮显示的模型列表 (保持不变)
highlight_models = ['5. Pruned FP32 (8L)', '6. Pruned FP16 (8L, GPU-Only)']


# ==============================================================================
# 6. *** 修改部分 ***：更新绘图函数
# ==============================================================================
def plot_radar_chart(df, metrics, title):
    # --- 新增逻辑：根据标题判断是否为CPU图，并修改颜色 ---
    # 创建一个颜色字典的副本，避免修改原始字典
    current_colors = copy.deepcopy(model_colors)
    if 'CPU' in title:
        # 如果是CPU图，将特定模型的颜色加深
        current_colors['3. INT8 PTQ (12L, CPU-Only)'] = '#016A4C' # 这是#029E73的加深版

    labels = [label.replace('(', '\n(').replace('Peak ', 'Peak\n') for label in metrics]
    num_vars = len(labels)
    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
    angles += angles[:1]
    
    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))

    # 遍历每个模型（每一行数据）并画线
    for _, row in df.iterrows():
        model_name = row['Model']
        # 使用当前图表对应的颜色字典
        color = current_colors.get(model_name, 'black') 
        data = row[metrics].values.flatten().tolist()
        data += data[:1]
        
        # --- 高亮视觉参数 (保持不变) ---
        if model_name in highlight_models:
            linewidth = 2.5
            line_alpha = 1.0
            fill_alpha = 0.25
            zorder = 10
        else:
            linewidth = 1.5
            line_alpha = 0.7
            fill_alpha = 0.1
            zorder = 5

        # 绘制线条和填充
        ax.plot(angles, data, color=color, linewidth=linewidth, alpha=line_alpha, label=model_name, zorder=zorder)
        ax.fill(angles, data, color=color, alpha=fill_alpha)

    # --- 美化图表 ---
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)
    ax.set_rlabel_position(0)
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(labels, fontsize=12)
    ax.tick_params(axis='x', which='major', pad=25)
    ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
    ax.set_yticklabels(["0.2", "0.4", "0.6", "0.8", "1.0"], color="grey", size=10)
    ax.set_ylim(0, 1.05)
    
    plt.title(title, size=20, color='black', y=1.15)
    
    # --- 布局修改：恢复为单行图例，并放置在图表外部 ---
    plt.legend(loc='upper right', bbox_to_anchor=(1.45, 1.1))
    
    # tight_layout() 会自动调整，这里不再需要手动subplots_adjust
    fig.tight_layout()
    
    output_filename = f'{title.replace(" ", "_")}.svg'
    # 使用 bbox_inches='tight' 来确保导出的SVG包含外部的图例
    plt.savefig(output_filename, bbox_inches='tight', format='svg', transparent=True)
    plt.close()
    print(f"图表已保存为: {output_filename}")


# ==============================================================================
# 7. 生成并保存图表
# ==============================================================================
print("正在生成GPU性能雷达图...")
plot_radar_chart(gpu_models_df, gpu_metrics, 'GPU 性能对比雷达图')

print("正在生成CPU性能雷达图...")
plot_radar_chart(cpu_models_df, cpu_metrics, 'CPU 性能对比雷达图')

print("\n所有图表已生成完毕！")